In [ ]:
##!pip install requests
import pandas as pd
import numpy as np
import requests

In [ ]:
raw_data  = {'size ': '10000'}
request = requests.get("https://imunizacao-es.saude.gov.br/_search?scroll=132m",auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))
#request.json()
#request.json()['hits']['hits']
df_minsaude= pd.json_normalize(data= request.json()['hits']['hits'])
#df_minsaude


In [ ]:
from scipy.stats import zscore
threshold =2
df_minsaude['zscore.paciente_idade'] = zscore(df_minsaude['_source.paciente_idade'])
df_minsaude['outliers'] = np.where((df_minsaude['zscore.paciente_idade'] - threshold > 0), True, np.where(df_minsaude['zscore.paciente_idade'] + threshold < 0, True, False))
df_minsaude.drop(df_minsaude[df_minsaude['outliers'] == True].index,inplace=True)

In [ ]:
df_minsaude =df_minsaude.drop(columns ='_index')
df_minsaude = df_minsaude.drop(columns ='_type')
df_minsaude["_source.estabelecimento_valor"]=df_minsaude["_source.estabelecimento_valor"].astype(int)#remoção de erro int64 ao fazer o marge
df_minsaude

,_id,_score,_source.estabelecimento_razaoSocial,_source.vacina_dataAplicacao,_source.vacina_grupoAtendimento_codigo,_source.estabelecimento_valor,_source.@timestamp,_source.sistema_origem,_source.vacina_lote,_source.id_sistema_origem,_source.estalecimento_noFantasia,_source.paciente_endereco_coIbgeMunicipio,_source.paciente_endereco_coPais,_source.estabelecimento_uf,_source.paciente_nacionalidade_enumNacionalidade,_source.paciente_endereco_nmPais,_source.paciente_idade,_source.paciente_racaCor_codigo,_source.vacina_codigo,_source.paciente_endereco_nmMunicipio,_source.estabelecimento_municipio_nome,_source.vacina_fabricante_referencia,_source.estabelecimento_municipio_codigo,_source.vacina_grupoAtendimento_nome,_source.document_id,_source.@version,_source.data_importacao_rnds,_source.paciente_endereco_cep,_source.paciente_dataNascimento,_source.vacina_descricao_dose,_source.vacina_fabricante_nome,_source.vacina_categoria_codigo,_source.paciente_endereco_uf,_source.vacina_categoria_nome,_source.redshift,_source.vacina_nome,_source.paciente_racaCor_valor,_source.paciente_id,_source.paciente_enumSexoBiologico,zscore.paciente_idade,outliers
0,4aa6ec29-3be5-46a1-8b0c-4fa109526629-i0b0,1.0,MINISTERIO DA SAUDE,2021-03-02T00:00:00.000Z,000920,2269880,2021-07-09T18:39:00.920Z,Novo PNI,200280,16341,MS HGB HOSPITAL GERAL DE BONSUCESSO,230440,10,RJ,B,BRASIL,54,01,86,FORTALEZA,RIO DE JANEIRO,Organization/61189445000156,330455,Recepcionista,4aa6ec29-3be5-46a1-8b0c-4fa109526629-i0b0,1,2021-03-19T16:38:38.000Z,60055,1966-12-17,2ª Dose,FUNDACAO BUTANTAN,9,CE,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,3d62c8945b3587ec3091be8345a52b936967afdc338236...,F,-1.580000,False
1,34cb9c47-5cf5-4073-80e8-edaaa26845d5-i0b0,1.0,SECRETARIA MUNICIPAL DE SAUDE,2021-01-28T00:00:00.000Z,000926,6449409,2021-07-09T18:39:00.920Z,Novo PNI,202010036,16341,SECRETARIA MUNICIPAL DE SAUDE DE GOIANIA,520870,10,GO,B,BRASIL,54,99,86,GOIANIA,GOIANIA,Organization/61189445000156,520870,Outros,34cb9c47-5cf5-4073-80e8-edaaa26845d5-i0b0,1,2021-03-19T15:15:47.000Z,74115,1966-12-28,1ª Dose,FUNDACAO BUTANTAN,9,GO,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,SEM INFORMACAO,a819f51aaa3235d0addb8efa59507af4e01a6e206f9532...,M,-1.580000,False
2,f9c263a1-c92b-406a-a6ca-bd1338c2b329-i0b0,1.0,MUNICIPIO DE COREMAS,2021-03-19T00:00:00.000Z,000203,3356337,2021-07-09T18:39:00.921Z,Novo PNI,210074,16341,UNIDADE DE SAUDE DA FAMILIA PSF II,250480,10,PB,B,BRASIL,74,01,86,COREMAS,COREMAS,Organization/61189445000156,250480,Pessoas de 70 a 74 anos,f9c263a1-c92b-406a-a6ca-bd1338c2b329-i0b0,1,2021-03-19T15:24:27.000Z,58770,1946-07-17,1ª Dose,FUNDACAO BUTANTAN,2,PB,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,bd064618eb1533c0a9f25e13776f1697cd2fb04526bf26...,M,0.091958,False
3,3b18ec6c-361a-4a19-bed5-b7956df60c2d-i0b0,1.0,MUNICIPIO DE CORNELIO PROCOPIO,2021-03-10T00:00:00.000Z,000205,7122446,2021-07-09T18:39:00.921Z,Novo PNI,210011A,16341,UBS MOACIR FERRACINI,410640,10,PR,B,BRASIL,83,01,86,CORNELIO PROCOPIO,CORNELIO PROCOPIO,Organization/61189445000156,410640,Pessoas de 80 anos ou mais,3b18ec6c-361a-4a19-bed5-b7956df60c2d-i0b0,1,2021-03-19T15:37:00.000Z,86300,1937-03-30,1ª Dose,FUNDACAO BUTANTAN,2,PR,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,804fd188d8687125d15d073e19a5ca74542de142b7a43d...,F,0.844339,False
4,da755c3c-c14e-4f87-9003-bf8ffca32c7e-i0b0,1.0,PREFEITURA MUNICIPAL DE PORTO ALEGRE,2021-03-19T00:00:00.000Z,000203,6883303,2021-07-09T18:39:00.921Z,Novo PNI,210051,16341,CLINICA DA FAMILIA IAPI,431490,10,RS,B,BRASIL,78,99,86,PORTO ALEGRE,PORTO ALEGRE,Organization/61189445000156,431490,Pessoas de 70 a 74 anos,da755c3c-c14e-4f87-9003-bf8ffca32c7e-i0b0,1,2021-03-19T16:43:07.000Z,90550,1942-06-23,1ª Dose,FUNDACAO BUTANTAN,2,RS,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,SEM INFORMACAO,cfa99267a8eb551735e574fdfa3805b1004d2bd6a7ce0c...,F,0.426349,False
5,02b329d8-1426-4994-8f58-f0175396f408-i0b0,1.0,PREFEITURA MUNICIPAL DE I

In [ ]:
url='https://sage.saude.gov.br/dados/repositorio/cadastro_estabelecimentos_cnes.zip'


In [ ]:
import requests
import io
import zipfile

response = requests.get(url)
arquivo=  zipfile.ZipFile(io.BytesIO(response.content))
csv=arquivo.extract(member='cadastro_estabelecimentos_cnes.csv')
csvpd = pd.read_csv(csv,sep=';')
csvpd["_source.estabelecimento_valor"] = csvpd["CNES"] 
#Após alguns testes, e após algumas analises do DataFrame, pode-se concluir, que o  CNES está respresentado  nas 2 bases de dados, sendo possivel
#realizar um margin left e acrecentar os dados adicionais dos postos de vacina.
csvpd['_source.estabelecimento_valor']=csvpd['_source.estabelecimento_valor'].astype(int) #remoção de erro int64 ao fazer o marge
csvpd


,CNES,UF,IBGE,NOME,LOGRADOURO,BAIRRO,LATITUDE,LONGITUDE,_source.estabelecimento_valor
0,33820,52,520170,UNIDADE DE SAUDE DA FAMILIA PSF 307,RUA H,NOVO MUNDO,"-15,90682","-52,22545",33820
1,108,26,260290,USF ALTO DOS INDIOS,RUA 17,PONTE DOS CARVALHOS,"-8,28389","-35,0321",108
2,116,26,260290,USF CHARNECA II,RUA 02,CHARNECA,"-8,28353","-35,02819",116
3,124,26,260290,USF SAO FRANCISCO I,RUA MANOEL DOMINGOS BARROS,SAO FRANCISCO,"-8,287","-35,035",124
4,132,26,260290,USF ROSARIO,RUA 01,ROSARIO,"-8,28389","-35,0321",132
...,...,...,...,...,...,...,...,...,...
41817,9718907,23,230780,JOAO ARISTON DIAS,SAO GERALDO,SAO GERALDO,"-3,12156","-40,06244",9718907
41818,9773274,33,330330,ESTETIC DENTE,RUA GAVIAO PEIXOTO,ICARAI,"-22,90744","-43,10604",9773274
41819,9776737,35,353320,UBS RACLE BARRETO DA SILVA,RUA JOAQUIM SERAFIM DE FREITAS,MORADA DO TREVO,"-21,1123","-51,49013",9776737
41820,9842330,24,241290,UNIDADE POTENGI,RUA AUGUSTO SEVERO,CENTRO,"-5,97503","-36,07464",9842330


In [ ]:
#result = df_minsaude.concat(right= csvpd,on='_source.estabelecimento_valor',how='left')
#result

result = df_minsaude.merge( csvpd, on=["_source.estabelecimento_valor"], how="left")
result

,_id,_score,_source.estabelecimento_razaoSocial,_source.vacina_dataAplicacao,_source.vacina_grupoAtendimento_codigo,_source.estabelecimento_valor,_source.@timestamp,_source.sistema_origem,_source.vacina_lote,_source.id_sistema_origem,_source.estalecimento_noFantasia,_source.paciente_endereco_coIbgeMunicipio,_source.paciente_endereco_coPais,_source.estabelecimento_uf,_source.paciente_nacionalidade_enumNacionalidade,_source.paciente_endereco_nmPais,_source.paciente_idade,_source.paciente_racaCor_codigo,_source.vacina_codigo,_source.paciente_endereco_nmMunicipio,_source.estabelecimento_municipio_nome,_source.vacina_fabricante_referencia,_source.estabelecimento_municipio_codigo,_source.vacina_grupoAtendimento_nome,_source.document_id,_source.@version,_source.data_importacao_rnds,_source.paciente_endereco_cep,_source.paciente_dataNascimento,_source.vacina_descricao_dose,_source.vacina_fabricante_nome,_source.vacina_categoria_codigo,_source.paciente_endereco_uf,_source.vacina_categoria_nome,_source.redshift,_source.vacina_nome,_source.paciente_racaCor_valor,_source.paciente_id,_source.paciente_enumSexoBiologico,zscore.paciente_idade,outliers,CNES,UF,IBGE,NOME,LOGRADOURO,BAIRRO,LATITUDE,LONGITUDE
0,4aa6ec29-3be5-46a1-8b0c-4fa109526629-i0b0,1.0,MINISTERIO DA SAUDE,2021-03-02T00:00:00.000Z,000920,2269880,2021-07-09T18:39:00.920Z,Novo PNI,200280,16341,MS HGB HOSPITAL GERAL DE BONSUCESSO,230440,10,RJ,B,BRASIL,54,01,86,FORTALEZA,RIO DE JANEIRO,Organization/61189445000156,330455,Recepcionista,4aa6ec29-3be5-46a1-8b0c-4fa109526629-i0b0,1,2021-03-19T16:38:38.000Z,60055,1966-12-17,2ª Dose,FUNDACAO BUTANTAN,9,CE,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,3d62c8945b3587ec3091be8345a52b936967afdc338236...,F,-1.580000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34cb9c47-5cf5-4073-80e8-edaaa26845d5-i0b0,1.0,SECRETARIA MUNICIPAL DE SAUDE,2021-01-28T00:00:00.000Z,000926,6449409,2021-07-09T18:39:00.920Z,Novo PNI,202010036,16341,SECRETARIA MUNICIPAL DE SAUDE DE GOIANIA,520870,10,GO,B,BRASIL,54,99,86,GOIANIA,GOIANIA,Organization/61189445000156,520870,Outros,34cb9c47-5cf5-4073-80e8-edaaa26845d5-i0b0,1,2021-03-19T15:15:47.000Z,74115,1966-12-28,1ª Dose,FUNDACAO BUTANTAN,9,GO,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,SEM INFORMACAO,a819f51aaa3235d0addb8efa59507af4e01a6e206f9532...,M,-1.580000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f9c263a1-c92b-406a-a6ca-bd1338c2b329-i0b0,1.0,MUNICIPIO DE COREMAS,2021-03-19T00:00:00.000Z,000203,3356337,2021-07-09T18:39:00.921Z,Novo PNI,210074,16341,UNIDADE DE SAUDE DA FAMILIA PSF II,250480,10,PB,B,BRASIL,74,01,86,COREMAS,COREMAS,Organization/61189445000156,250480,Pessoas de 70 a 74 anos,f9c263a1-c92b-406a-a6ca-bd1338c2b329-i0b0,1,2021-03-19T15:24:27.000Z,58770,1946-07-17,1ª Dose,FUNDACAO BUTANTAN,2,PB,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,bd064618eb1533c0a9f25e13776f1697cd2fb04526bf26...,M,0.091958,False,3356337.0,25.0,250480.0,UNIDADE DE SAUDE DA FAMILIA PSF II,SANTA RITA DE CASSIA,CENTRO,"-7,01909","-37,94532"
3,3b18ec6c-361a-4a19-bed5-b7956df60c2d-i0b0,1.0,MUNICIPIO DE CORNELIO PROCOPIO,2021-03-10T00:00:00.000Z,000205,7122446,2021-07-09T18:39:00.921Z,Novo PNI,210011A,16341,UBS MOACIR FERRACINI,410640,10,PR,B,BRASIL,83,01,86,CORNELIO PROCOPIO,CORNELIO PROCOPIO,Organization/61189445000156,410640,Pessoas de 80 anos ou mais,3b18ec6c-361a-4a19-bed5-b7956df60c2d-i0b0,1,2021-03-19T15:37:00.000Z,86300,1937-03-30,1ª Dose,FUNDACAO BUTANTAN,2,PR,Faixa Etária,v2,Covid-19-Coronavac-Sinovac/Butantan,BRANCA,804fd188d8687125d15d073e19a5ca74542de142b7a43d...,F,0.844339,False,7122446.0,41.0,410640.0,UBS MOACIR FERRACINI,TOMIO TAKAHASHI,JARDIM SANTA ROSA,"-23,181","-50,647"
4,da755c3c-c14e-4f87-9003-bf8ffca32c7e-i0b0,1.0,PREFEITURA MUNICIPAL DE PORTO ALEGRE,2021-03-19T00:00:00.000Z,000203,6883303,2021-07-09T18:39:00.921Z,Novo PNI,210051,16341,CLINICA DA FAMILIA IAPI,431490,10,RS,B,BRASIL,78,99,86,PORTO ALEGRE,PORTO ALEGRE,Organization/61189445000156,431490,Pessoas de 70 a 74 a

In [313]:
a=result.to_csv('base_vacinados.csv',header=False,sep=';')
  

In [314]:
b=result.to_excel('base_vacinados.xlsx', index = False)
